GitHub link: https://github.com/rohanneps/deakin/

Welcome to your assignment this week! 

To better understand bias and discrimination in AI, in this assignment, we will look at a Natural Language Processing use case. 

Natural Language Pocessing (NLP) is a branch of Artificial Intelligence (AI) that helps computers to understand, to interpret and to manipulate natural (i.e. human) language.
Imagine NLP-powered machines as black boxes that are capable of understanding and evaluating the context of the input documents (i.e. collection of words), outputting meaningful results that depend on the task the machine is designed for.


![](images/1_3zMvUnPzYZF9CSHdj6hT5A.png "the title")



<caption><center> Documents are fed into magic NLP model capable to get, for instance, the sentiment of the original content</center></caption>



Just like any other machine learning algorithm, biased data results in biased outcomes. And just like any other algorithm, results debiasing is painfully annoying, to the point that it might be simpler to unbias the society itself.

# The big deal: word embeddings

Words must be represented as **numeric vectors** in order to be fed into machine learning algorithms. One of the most powerful (and popular) ways to do it is through **Word Embeddings**. In word embedding models, each word in a given language is assigned to a high-dimensional vector, such that **the geometry of the vectors captures relations between the words**. 


Because word embeddings are very computionally expensive to train, most ML practitioners will load a pre-trained set of embeddings. 

**After this assignment you will be able to:**

- Load pre-trained word vectors, and measure similarity using cosine similarity
- Use word embeddings to solve word analogy problems such as Man is to Woman as King is to ______. 
- Modify word embeddings to reduce their gender bias 

Let's get started! Run the following cell to install all the packages you will need.

In [ ]:
#!pip install numpy
#!pip install keras
#!pip install tensorflow

Run the following cell to load the packages you will need.

In [2]:
import numpy as np
from w2v_utils import *

tf.get_logger().setLevel('INFO')

Next, lets load the word vectors. For this assignment, we will use 50-dimensional GloVe vectors to represent words. Run the following cell to load the `word_to_vec_map`. 

In [3]:
words, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

You've loaded:
- `words`: set of words in the vocabulary.
- `word_to_vec_map`: dictionary mapping words to their GloVe vector representation.



In [4]:
print("Example of words: ",list(words)[:10])
print("Vector for word 'person' = ", word_to_vec_map.get('person'))

Example of words:  ['gumilyov', 'thingy', 'hyo', '125-cc', 'waif', 'jabarin', 'time-resolved', '42.20', 'patched', 'summarize']
Vector for word 'person' =  [ 0.61734    0.40035    0.067786  -0.34263    2.0647     0.60844
  0.32558    0.3869     0.36906    0.16553    0.0065053 -0.075674
  0.57099    0.17314    1.0142    -0.49581   -0.38152    0.49255
 -0.16737   -0.33948   -0.44405    0.77543    0.20935    0.6007
  0.86649   -1.8923    -0.37901   -0.28044    0.64214   -0.23549
  2.9358    -0.086004  -0.14327   -0.50161    0.25291   -0.065446
  0.60768    0.13984    0.018135  -0.34877    0.039985   0.07943
  0.39318    1.0562    -0.23624   -0.4194    -0.35332   -0.15234
  0.62158    0.79257  ]


GloVe vectors provide much more useful information about the meaning of individual words. Lets now see how you can use GloVe vectors to decide how similar two words are. 


# Cosine similarity

To measure how similar two words are, we need a way to measure the degree of similarity between two embedding vectors for the two words. Given two vectors $u$ and $v$, cosine similarity is defined as follows: 

\begin{equation}
\text{CosineSimilarity(u, v)} = \frac {u . v} {||u||_2 ||v||_2} = cos(\theta) \tag{1}
\end{equation}

where $u.v$ is the dot product (or inner product) of two vectors, $||u||_2$ is the norm (or length) of the vector $u$, and $\theta$ is the angle between $u$ and $v$. This similarity depends on the angle between $u$ and $v$. If $u$ and $v$ are very similar, their cosine similarity will be close to 1; if they are dissimilar, the cosine similarity will take a smaller value. 


![](images/cosine_sim.png)

<caption><center> <b>Figure 1</b>: The cosine of the angle between two vectors is a measure of how similar they are</center></caption>



***

**Task 1**: Implement the function `cosine_similarity()` to evaluate similarity between word vectors.

**Reminder**: The norm of $u$ is defined as $||u||_2 = \sqrt{\sum_{i=1}^{n} u_i^2}$

In [5]:
## Task 1
# cosine_similarity

def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similariy between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    ## START YOU CODE HERE
    # since both the vectors have to be of same size
    if len(u)!=len(v):
        return 'The length of both vectors should be same'
    # placeholders
    dot_prod = 0
    magnitude_u = 0
    magnitude_v = 0
    for idx in range(0, len(u)):
        dot_prod += u[idx]*v[idx]
        magnitude_u += u[idx]**2
        magnitude_v += v[idx]**2
    # Square rooting
    magnitude_u = magnitude_u**(1/2)
    magnitude_v = magnitude_v**(1/2)
    # computing cosine similarity
    cos_sim = dot_prod/(magnitude_u*magnitude_v)    
    return cos_sim
    ## END

***

**Task 2**: Implement `most_similar_word` which returns the most similar word to a word.

In [60]:
## Task 2
# GRADED FUNCTION: most_similar_word

def most_similar_word(word, word_to_vec_map):
    """
    Most similar word return the most similar word to the word u.
        
    Arguments:
        word -- a word, string        
        word_to_vec_map -- dictionary that maps words to their corresponding vectors. 
    
    Returns:
    best_word --  the most similar word to u as measured by cosine similarity
    """
    ## START YOU CODE HERE
    # placeholders to get the most similar word
    most_similar_word = None
    max_cosine_similarity = -1
    word = word.lower()
    # getting the vector embedding of the word to search
    search_word_embedding = word_to_vec_map[word]
    # iterating through each word in the word_embedding dictionary
    for vec_word in word_to_vec_map:
        if word != vec_word:
            # comparing to other words apart from itself
            vec_word_embedding = word_to_vec_map[vec_word]
            cosine_sim = cosine_similarity(search_word_embedding, vec_word_embedding)
            if cosine_sim>max_cosine_similarity:
                # the cosine similarity will be close to 1 for vectors that are similar
                max_cosine_similarity = cosine_sim
                most_similar_word = vec_word
    print(max_cosine_similarity)
    return most_similar_word

    ## END


Answer the questions below:

**TASK 3: Write a code the answer the following questions:**

***

What is the similarity between the words brother and friend?

In [6]:
cosine_sim = cosine_similarity(word_to_vec_map['brother'], word_to_vec_map['friend'])
print('Similarity between the words brother and friend is: {}'.format(cosine_sim))

Similarity between the words brother and friend is: 0.8713178668124654


What is the similarity between the words computer and kid?

In [7]:
cosine_sim = cosine_similarity(word_to_vec_map['computer'], word_to_vec_map['kid'])
print('Similarity between the words computer and kid is: {}'.format(cosine_sim))

Similarity between the words computer and kid is: 0.43800166210363856


What is the similarity between the words V1=(france - paris) and V2=(rome - italy)?

In [8]:
# vector subtractions
v1 = word_to_vec_map['france'] - word_to_vec_map['paris']
v2 = word_to_vec_map['rome'] - word_to_vec_map['italy']

cosine_sim = cosine_similarity(v1, v2)
print('Similarity between the words v1 and v2 is: {}'.format(cosine_sim))

Similarity between the words v1 and v2 is: -0.6751479308174202


What is the most similar word to computer?


In [56]:
similar_word = most_similar_word('computer', word_to_vec_map)
print('Most similar word to computer is: {}'.format(similar_word))

0.9165044765653498
Most similar word to computer is: computers


What is the most similar word to australia?

In [57]:
similar_word = most_similar_word('australia', word_to_vec_map)
print('Most similar word to australia is: {}'.format(similar_word))

0.9207795062284202
Most similar word to australia is: zealand


What is the most similar word to python?


In [58]:
similar_word = most_similar_word('python', word_to_vec_map)
print('Most similar word to python is: {}'.format(similar_word))

0.6916364915886599
Most similar word to python is: reticulated


***

Playing around the cosine similarity of other inputs will give you a better sense of how word vectors behave. 

#  Word analogy task

In the word analogy task, we complete the sentence <font color='brown'>"*a* is to *b* as *c* is to **____**"</font>. An example is <font color='brown'> '*man* is to *woman* as *king* is to *queen*' </font>. In detail, we are trying to find a word *d*, such that the associated word vectors $e_a, e_b, e_c, e_d$ are related in the following manner: $e_b - e_a \approx e_d - e_c$. We will measure the similarity between $e_b - e_a$ and $e_d - e_c$ using cosine similarity. 


**Task 4**: Complete the code below to be able to perform word analogies!

***

In [156]:
## Task 4
# GRADED FUNCTION: complete_analogy

def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    """
    Performs the word analogy task as explained above: a is to b as c is to ____. 
    
    Arguments:
    word_a -- a word, string
    word_b -- a word, string
    word_c -- a word, string
    word_to_vec_map -- dictionary that maps words to their corresponding vectors. 
    
    Returns:
    best_word --  the word such that v_b - v_a is close to v_best_word - v_c, as measured by cosine similarity
    """
    ## START YOU CODE HERE
    # converting to lower
    word_a,word_b,word_c = word_a.lower(),word_b.lower(),word_c.lower()
    # getting the word embeddings
    word_a_embedding = word_to_vec_map[word_a]
    word_b_embedding = word_to_vec_map[word_b]
    word_c_embedding = word_to_vec_map[word_c]
    # e_d = e_c + e_b - e_a
    difference_embedding = word_c_embedding + word_b_embedding - word_a_embedding
    # placeholders
    most_similar_word = None
    max_cosine_similarity = -1

    for vec_word in word_to_vec_map:
        if vec_word != word_b:
            vec_word_embedding = word_to_vec_map[vec_word]
            # the target word needs to have embedding most similar to the above equation embedding
            cosine_sim = cosine_similarity(difference_embedding, vec_word_embedding)
            if cosine_sim>max_cosine_similarity:
                # the cosine similarity will be close to 1 for vectors that are similar
                max_cosine_similarity = cosine_sim
                most_similar_word = vec_word
            
    return most_similar_word
    ##END


Run the cell below to test your code, this may take 1-2 minutes.

In [70]:
triads_to_try = [('italy', 'italian', 'spain'), ('india', 'delhi', 'japan'), ('man', 'woman', 'boy'), ('small', 'smaller', 'large')]
for triad in triads_to_try:
    print ('{} -> {} :: {} -> {}'.format( *triad, complete_analogy(*triad,word_to_vec_map)))

italy -> italian :: spain -> spanish
india -> delhi :: japan -> tokyo
man -> woman :: boy -> girl
small -> smaller :: large -> smaller


In [157]:
complete_analogy('small','smaller','big', word_to_vec_map)

'big'

In [158]:
complete_analogy('man','computer','woman', word_to_vec_map)

'computers'

Once you get the correct expected output, please feel free to modify the input cells above to test your own analogies. Try to find some other analogy pairs that do work, but also find some where the algorithm doesn't give the right answer: For example, you can try small->smaller as big->?.  

# Debiasing word vectors

In the following exercise, you will examine gender biases that can be reflected in a word embedding, and explore algorithms for reducing the bias. In addition to learning about the topic of debiasing, this exercise will also help hone your intuition about what word vectors are doing. This section involves a bit of linear algebra, though you can probably complete it even without being expert in linear algebra, and we encourage you to give it a shot. 

Lets first see how the GloVe word embeddings relate to gender. You will first compute a vector $g = e_{woman}-e_{man}$, where $e_{woman}$ represents the word vector corresponding to the word *woman*, and $e_{man}$ corresponds to the word vector corresponding to the word *man*. The resulting vector $g$ roughly encodes the concept of "gender". (You might get a more accurate representation if you compute $g_1 = e_{mother}-e_{father}$, $g_2 = e_{girl}-e_{boy}$, etc. and average over them. But just using $e_{woman}-e_{man}$ will give good enough results for now.) 


**Task 5:** Compute the bias vector using woman - man

***

In [17]:
## START YOU CODE HERE

g = word_to_vec_map['woman']-word_to_vec_map['man']

## END

print(g)

[-0.087144    0.2182     -0.40986    -0.03922    -0.1032      0.94165
 -0.06042     0.32988     0.46144    -0.35962     0.31102    -0.86824
  0.96006     0.01073     0.24337     0.08193    -1.02722    -0.21122
  0.695044   -0.00222     0.29106     0.5053     -0.099454    0.40445
  0.30181     0.1355     -0.0606     -0.07131    -0.19245    -0.06115
 -0.3204      0.07165    -0.13337    -0.25068714 -0.14293    -0.224957
 -0.149       0.048882    0.12191    -0.27362    -0.165476   -0.20426
  0.54376    -0.271425   -0.10245    -0.32108     0.2516     -0.33455
 -0.04371     0.01258   ]


***

Now, you will consider the cosine similarity of different words with $g$. Consider what a positive value of similarity means vs a negative cosine similarity. 

**Task 6: Compute and print the similarity between g and the words in name_list**

***
    

In [18]:
print ('List of names and their similarities with constructed vector:')
name_list = ['john', 'marie', 'sophie', 'ronaldo', 'priya', 'rahul', 'danielle', 'reza', 'katy', 'yasmin']

## START YOU CODE HERE
for name in name_list:
    name_embedding = word_to_vec_map[name]
    cosine_sim = cosine_similarity(g, name_embedding)
    print('Cosine similarity for {} with g is {}'.format(name, cosine_sim))
## END 

List of names and their similarities with constructed vector:
Cosine similarity for john with g is -0.2316335614597372
Cosine similarity for marie with g is 0.3155979353960729
Cosine similarity for sophie with g is 0.31868789859418784
Cosine similarity for ronaldo with g is -0.31244796850329426
Cosine similarity for priya with g is 0.176320418390094
Cosine similarity for rahul with g is -0.1691547103923171
Cosine similarity for danielle with g is 0.24393299216283895
Cosine similarity for reza with g is -0.07930429672199547
Cosine similarity for katy with g is 0.28310686595726137
Cosine similarity for yasmin with g is 0.23313857767928758


***

**TASK 7: What do you observe?**

***

The embeddings of bias vector g is more similar to names which are of female compared to names which are males. The cosine similarity between g and female names are closer to 1 compared to cosine similarity between g and names of male which are negative. Event though vector g is the generalized concept of gender we can see that this vector is biased in the making. This means that names normally associated with female have similarity with this generalized vector g.


***

**Task 8: Compute and print the similarity between g and the words in word_list:**

***

In [19]:
print('Other words and their similarities:')
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 'receptionist', 
             'technology',  'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']

## START YOU CODE HERE
for word in word_list:
    word_embedding = word_to_vec_map[word]
    cosine_sim = cosine_similarity(g, word_embedding)
    print('Cosine similarity for {} with g is {}'.format(word, cosine_sim))

## END 


Other words and their similarities:
Cosine similarity for lipstick with g is 0.2769191625638266
Cosine similarity for guns with g is -0.18884855678988977
Cosine similarity for science with g is -0.060829065409296966
Cosine similarity for arts with g is 0.00818931238588036
Cosine similarity for literature with g is 0.06472504433459929
Cosine similarity for warrior with g is -0.2092016464112528
Cosine similarity for doctor with g is 0.11895289410935035
Cosine similarity for tree with g is -0.07089399175478088
Cosine similarity for receptionist with g is 0.33077941750593737
Cosine similarity for technology with g is -0.1319373244755429
Cosine similarity for fashion with g is 0.03563894625772697
Cosine similarity for teacher with g is 0.17920923431825658
Cosine similarity for engineer with g is -0.0803928049452407
Cosine similarity for pilot with g is 0.0010764498991917163
Cosine similarity for computer with g is -0.103303588738505
Cosine similarity for singer with g is 0.1850051813649628


In [20]:
print('Other words and their similarities:')
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 'receptionist', 
             'technology',  'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']

## START YOU CODE HERE
for word in word_list:
    word_embedding = word_to_vec_map[word]
    cosine_sim = cosine_similarity(word_to_vec_map['man']-word_to_vec_map['woman'], word_embedding)
    print('Cosine similarity for {} with g compliment is {}'.format(word, cosine_sim))

## END 


Other words and their similarities:
Cosine similarity for lipstick with g compliment is -0.2769191625638266
Cosine similarity for guns with g compliment is 0.18884855678988977
Cosine similarity for science with g compliment is 0.060829065409296966
Cosine similarity for arts with g compliment is -0.00818931238588036
Cosine similarity for literature with g compliment is -0.06472504433459929
Cosine similarity for warrior with g compliment is 0.2092016464112528
Cosine similarity for doctor with g compliment is -0.11895289410935035
Cosine similarity for tree with g compliment is 0.07089399175478088
Cosine similarity for receptionist with g compliment is -0.33077941750593737
Cosine similarity for technology with g compliment is 0.1319373244755429
Cosine similarity for fashion with g compliment is -0.03563894625772697
Cosine similarity for teacher with g compliment is -0.17920923431825658
Cosine similarity for engineer with g compliment is 0.0803928049452407
Cosine similarity for pilot with g

***

**TASK 7: What do you observe?**

***

Similarly, here we can see that the words 'reception' and 'lipstick' have high similarity with the generalized gender embedding g. Similarly gender-independent objects such as computer, tree, guns do not share similarity to the embedding g enough though this vector is assumed to be gender-independent. Futhermore occupation such as teaching and singing are also more associated with embedding g.
This means that object and occupations stereotyped as being associated with females in our socities are also encoded in the training data for GloVe and its resulting biased embeddings.

***



We'll see below how to reduce the bias of these vectors, using an algorithm due to [Boliukbasi et al., 2016](https://arxiv.org/abs/1607.06520). Note that some word pairs such as "actor"/"actress" or "grandmother"/"grandfather" should remain gender specific, while other words such as "receptionist" or "technology" should be neutralized, i.e. not be gender-related. You will have to treat these two type of words differently when debiasing.

# Neutralize bias for non-gender specific words 

The figure below should help you visualize what neutralizing does. If you're using a 50-dimensional word embedding, the 50 dimensional space can be split into two parts: The bias-direction $g$, and the remaining 49 dimensions, which we'll call $g_{\perp}$. In linear algebra, we say that the 49 dimensional $g_{\perp}$ is perpendicular (or "orthogonal") to $g$, meaning it is at 90 degrees to $g$. The neutralization step takes a vector such as $e_{receptionist}$ and zeros out the component in the direction of $g$, giving us $e_{receptionist}^{debiased}$. 

Even though $g_{\perp}$ is 49 dimensional, given the limitations of what we can draw on a screen, we illustrate it using a 1 dimensional axis below. 


![](images/neutral.png)


<caption><center> <b>Figure 2</b>: The word vector for "receptionist" represented before and after applying the neutralize operation. </center></caption>

**TASK 10**: Implement `neutralize()` to remove the bias of words such as "receptionist" or "scientist". Given an input embedding $e$, you can use the following formulas to compute $e^{debiased}$: 


\begin{equation}
e^{bias\_component} = \frac{e \cdot g}{||g||_2^2} * g\tag{2}
\end{equation}

\begin{equation}
e^{debiased} = e - e^{bias\_component}\tag{3}
\end{equation}

If you are an expert in linear algebra, you may recognize $e^{bias\_component}$ as the projection of $e$ onto the direction $g$. If you're not an expert in linear algebra, don't worry about this.

<!-- 
**Reminder**: a vector $u$ can be split into two parts: its projection over a vector-axis $v_B$ and its projection over the axis orthogonal to $v$:
$$u = u_B + u_{\perp}$$
where : $u_B = $ and $ u_{\perp} = u - u_B $
!--> 
***

In [137]:
def get_biased_component(embedding, g):
    """
    Compute the embeddings for the biased component. In other words the embeddings of projection of e on the bias axis g.
    
    Arguments:
        embedding -- vector embedding
        g -- numpy-array of shape (50,), corresponding to the bias axis (such as gender)
    
    Returns:
        e_debiased -- neutralized word vector representation of the input "embedding"
    """
    # magnitude of vector squared is same as dot product of vector with iteself
    # square of norm-2 is same as dot product of vector with itself
    scalar_component = np.dot(embedding, g)/np.dot(g, g)  # coefficient in equation(2)
    e_bias_component = scalar_component*g           # eq(2)
    return e_bias_component

In [139]:
# TASK 10
# GRADED neutralize

def neutralize(word, g, word_to_vec_map):
    """
    Removes the bias of "word" by projecting it on the space orthogonal to the bias axis. 
    This function ensures that gender neutral words are zero in the gender subspace.
    
    Arguments:
        word -- string indicating the word to debias
        g -- numpy-array of shape (50,), corresponding to the bias axis (such as gender)
        word_to_vec_map -- dictionary mapping words to their corresponding vectors.
    
    Returns:
        e_debiased -- neutralized word vector representation of the input "word"
    """
    ## START YOU CODE HERE

    word_embedding = word_to_vec_map[word]  
    e_bias_component = get_biased_component(word_embedding, g)
    e_debiased = word_embedding-e_bias_component
    return e_debiased
    ## END 
    

In [140]:
e = "receptionist"
print("cosine similarity between " + e + " and g, before neutralizing: ", cosine_similarity(word_to_vec_map["receptionist"], g))

e_debiased = neutralize("receptionist", g, word_to_vec_map)
print("cosine similarity between " + e + " and g, after neutralizing: ", cosine_similarity(e_debiased, g))

cosine similarity between receptionist and g, before neutralizing:  0.33077941750593737
cosine similarity between receptionist and g, after neutralizing:  -7.429062997572218e-17


# Equalization algorithm for gender-specific words

Next, lets see how debiasing can also be applied to word pairs such as "actress" and "actor." Equalization is applied to pairs of words that you might want to have differ only through the gender property. As a concrete example, suppose that "actress" is closer to "babysit" than "actor." By applying neutralizing to "babysit" we can reduce the gender-stereotype associated with babysitting. But this still does not guarantee that "actor" and "actress" are equidistant from "babysit." The equalization algorithm takes care of this. 

The key idea behind equalization is to make sure that a particular pair of words are equi-distant from the 49-dimensional $g_\perp$. The equalization step also ensures that the two equalized steps are now the same distance from $e_{receptionist}^{debiased}$, or from any other word that has been neutralized. In pictures, this is how equalization works: 


![](images/equalize10.png)



The derivation of the linear algebra to do this is a bit more complex. (See Bolukbasi et al., 2016 for details.) But the key equations are: 

\begin{equation}
\mu = \frac{e_{w1} + e_{w2}}{2}\tag{4}
\end{equation}

\begin{equation}
\mu_{B} = \frac {\mu \cdot bias\_axis}{||bias\_axis||_2^2} *bias\_axis
\tag{5}
\end{equation}

\begin{equation}
\mu_{\perp} = \mu - \mu_{B} \tag{6}
\end{equation}

\begin{equation}
e_{w1B} = \frac {e_{w1} \cdot bias\_axis}{||bias\_axis||_2^2} *bias\_axis
\tag{7}
\end{equation}

\begin{equation}
e_{w2B} = \frac {e_{w2} \cdot bias\_axis}{||bias\_axis||_2^2} *bias\_axis
\tag{8}
\end{equation}

\begin{equation}
e_{w1B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w1B}} - \mu_B} {||(e_{w1} - \mu_{\perp}) - \mu_B||} \tag{9}
\end{equation}

\begin{equation}
e_{w2B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w2B}} - \mu_B} {||(e_{w2} - \mu_{\perp}) - \mu_B||} \tag{10}
\end{equation}

\begin{equation}
e_1 = e_{w1B}^{corrected} + \mu_{\perp} \tag{11}
\end{equation}

\begin{equation}
e_2 = e_{w2B}^{corrected} + \mu_{\perp} \tag{12}
\end{equation}

**TASK 11**: Implement the function below. Use the equations above to get the final equalized version of the pair of words. Good luck!

***

In [152]:
# TASK 11
# GRADED equalize

def equalize(pair, bias_axis, word_to_vec_map):
    """
    Debias gender specific words by following the equalize method described in the figure above.
    
    Arguments:
    pair -- pair of strings of gender specific words to debias, e.g. ("actress", "actor") 
    bias_axis -- numpy-array of shape (50,), vector corresponding to the bias axis, e.g. gender
    word_to_vec_map -- dictionary mapping words to their corresponding vectors
    
    Returns
    e_1 -- word vector corresponding to the first word
    e_2 -- word vector corresponding to the second word
    """
    ## START YOU CODE HERE
    word1, word2 = pair
    # get embeddings for the pair
    e_w1 = word_to_vec_map[word1]
    e_w2 = word_to_vec_map[word2]
    
    # computing equations
    mu = (e_w1+e_w2)/2.0                             # eq(4)
    
    mu_B = get_biased_component(mu, bias_axis)     # eq(5)
    mu_perpen = mu - mu_B                          # eq(6)
    
    e_w1_B = get_biased_component(e_w1, bias_axis) # eq(7)
    e_w2_B = get_biased_component(e_w2, bias_axis) # eq(8)
    
    # coefficients for eq(9) and (10)
    # dot product of vector with itself is the squared of vector norm-2
    e_corrected_coeff = (abs(1 - np.dot(mu_perpen, mu_perpen)))**(1/2)
    
    e_w1_B_corrected = e_corrected_coeff*((e_w1_B - mu_B)/abs(e_w1 - mu_perpen - mu_B))        # eq(9)
    e_w2_B_corrected = e_corrected_coeff*((e_w2_B - mu_B)/abs(e_w2 - mu_perpen - mu_B))        # eq(10)
    
    e_1 = e_w1_B_corrected + mu_perpen             # eq(11)
    e_2 = e_w2_B_corrected + mu_perpen             # eq(12)
    
    return e_1, e_2
    ##END

In [149]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"man\"], gender) = ", cosine_similarity(word_to_vec_map["man"], g))
print("cosine_similarity(word_to_vec_map[\"woman\"], gender) = ", cosine_similarity(word_to_vec_map["woman"], g))
print()
e1, e2 = equalize(("man", "woman"), g, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g))

cosine similarities before equalizing:
cosine_similarity(word_to_vec_map["man"], gender) =  -0.11711095765336828
cosine_similarity(word_to_vec_map["woman"], gender) =  0.3566661884627036

cosine similarities after equalizing:
cosine_similarity(e1, gender) =  -0.7165727525843932
cosine_similarity(e2, gender) =  0.7396596474928907


In [154]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"actor\"], gender) = ", cosine_similarity(word_to_vec_map["actor"], g))
print("cosine_similarity(word_to_vec_map[\"actress\"], gender) = ", cosine_similarity(word_to_vec_map["actress"], g))
print()
e1, e2 = equalize(("actor", "actress"), g, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g))

cosine similarities before equalizing:
cosine_similarity(word_to_vec_map["actor"], gender) =  -0.08387555382505694
cosine_similarity(word_to_vec_map["actress"], gender) =  0.33422494897899774

cosine similarities after equalizing:
cosine_similarity(e1, gender) =  -0.5221008397426838
cosine_similarity(e2, gender) =  0.5236317933422892


In [150]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"father\"], gender) = ", cosine_similarity(word_to_vec_map["father"], g))
print("cosine_similarity(word_to_vec_map[\"mother\"], gender) = ", cosine_similarity(word_to_vec_map["mother"], g))
print()
e1, e2 = equalize(("father", "mother"), g, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g))

cosine similarities before equalizing:
cosine_similarity(word_to_vec_map["father"], gender) =  0.034164374780686756
cosine_similarity(word_to_vec_map["mother"], gender) =  0.34016319002265666

cosine similarities after equalizing:
cosine_similarity(e1, gender) =  -0.18147328378900485
cosine_similarity(e2, gender) =  0.18163543434759405


In [153]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"man\"], e_debiased) = ", cosine_similarity(word_to_vec_map["man"], e_debiased))
print("cosine_similarity(word_to_vec_map[\"woman\"], e_debiased) = ", cosine_similarity(word_to_vec_map["woman"], e_debiased))
print()
e1, e2 = equalize(("man", "woman"), e_debiased, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, e_debiased) = ", cosine_similarity(e1, e_debiased))
print("cosine_similarity(e2, e_debiased) = ", cosine_similarity(e2, e_debiased))

cosine similarities before equalizing:
cosine_similarity(word_to_vec_map["man"], e_debiased) =  0.36998963140209173
cosine_similarity(word_to_vec_map["woman"], e_debiased) =  0.3480511103205663

cosine similarities after equalizing:
cosine_similarity(e1, e_debiased) =  -1.2455431719824582e-17
cosine_similarity(e2, e_debiased) =  -1.4878013189330454e-14


**TASK 12: What do you observe?**

***

From the above example we can observer that the generalized encoded notion of gender 'g' reduced the gender
stereotypes in the word embedding. Male nouns such as 'male', 'actor' has moved to similar distance to eh female nouns such as 'woman', 'actress'. Similarly the neutralized word 'receptionist' whose gender-bias has been removed is now equi-distance from male and female noune as shown above.


***

Please feel free to play with the input words in the cell above, to apply equalization to other pairs of words. 

These debiasing algorithms are very helpful for reducing bias, but are not perfect and do not eliminate all traces of bias. For example, one weakness of this implementation was that the bias direction $g$ was defined using only the pair of words _woman_ and _man_. As discussed earlier, if $g$ were defined by computing $g_1 = e_{woman} - e_{man}$; $g_2 = e_{mother} - e_{father}$; $g_3 = e_{girl} - e_{boy}$; and so on and averaging over them, you would obtain a better estimate of the "gender" dimension in the 50 dimensional word embedding space. Feel free to play with such variants as well.  
                     

# Congratulations!

You've come to the end of this assignment, and have seen a lot of the ways that word vectors can be used as well as  modified. Here are the main points you should remember:

- Cosine similarity a good way to compare similarity between pairs of word vectors. (Though L2 distance works too.) 
- For NLP applications, using a pre-trained set of word vectors from the internet is often a good way to get started. 
- Bias in data is an important problem.
- Neutralize and equalize allow to reduce bias in the data.

Congratulations on finishing this notebook! 



# References
- The debiasing algorithm is from Bolukbasi et al., 2016, [Man is to Computer Programmer as Woman is to
Homemaker? Debiasing Word Embeddings](https://papers.nips.cc/paper/6228-man-is-to-computer-programmer-as-woman-is-to-homemaker-debiasing-word-embeddings.pdf)
- The GloVe word embeddings were due to Jeffrey Pennington, Richard Socher, and Christopher D. Manning. (https://nlp.stanford.edu/projects/glove/)
